In [1]:
%cd ..

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark


In [2]:
# this just to make sure we are using only on CPU
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg
import os.path as op
import time

from keras.callbacks import TensorBoard, ModelCheckpoint
import numpy as np
from keras_tqdm import TQDMNotebookCallback
from tqdm import tqdm_notebook
import tensorflow as tf

from fastmri_recon.models.unet import unet
from fastmri_recon.data.oasis_sequences import ZeroFilled2DSequence

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.



                 .|'''|       /.\      '||'''|,
                 ||          // \\      ||   ||
'||''|, '||  ||` `|'''|,    //...\\     ||...|'
 ||  ||  `|..||   .   ||   //     \\    ||
 ||..|'      ||   |...|' .//       \\. .||
 ||       ,  |'
.||        ''

Package version: 0.0.3

License: CeCILL-B

Authors: 

Antoine Grigis <antoine.grigis@cea.fr>
Samuel Farrens <samuel.farrens@cea.fr>
Jean-Luc Starck <jl.stark@cea.fr>
Philippe Ciuciu <philippe.ciuciu@cea.fr>

Dependencies: 

scipy          : >=1.3.0   - required | 1.3.0     installed
numpy          : >=1.16.4  - required | 1.16.4    installed
matplotlib     : >=3.0.0   - required | 3.1.0     installed
astropy        : >=3.0.0   - required | 2.0.8     installed
nibabel        : >=2.3.2   - required | 2.4.1     installed
pyqtgraph      : >=0.10.0  - required | 0.10.0    installed
progressbar2   : >=3.34.3  - required | ?         installed
modopt         : >=1.4.0   - required | 1.3.3     installed
scikit-learn   : >=0.19.1  - requi

In [4]:
# paths
train_path = '/media/Zaccharie/UHRes/OASIS_data/'

In [5]:
# generators
AF = 4
train_gen = ZeroFilled2DSequence(
    train_path, 
    af=AF, 
    inner_slices=32, 
    scale_factor=1e-2, 
    seed=0, 
    val_split=0.1,
)
val_gen = train_gen.val_sequence

# train_gen.filenames = train_gen.filenames[:5]
# val_gen.filenames = val_gen.filenames[:1]

In [6]:
run_params = {
    'n_layers': 4, 
    'pool': 'max', 
    "layers_n_channels": [16, 32, 64, 128], 
    'layers_n_non_lins': 2,
#     'n_layers': 2,
}
n_epochs = 350
run_id = f'unet_af{AF}_oasis_{int(time.time())}'
chkpt_path = f'checkpoints/{run_id}' + '-{epoch:02d}.hdf5'
print(run_id)

unet_af4_oasis_1568899762


In [7]:
chkpt_cback = ModelCheckpoint(chkpt_path, period=500)
log_dir = op.join('logs', run_id)
tboard_cback = TensorBoard(
    log_dir=log_dir, 
    histogram_freq=0, 
    write_graph=True, 
    write_images=False, 
)
# fix from https://github.com/bstriner/keras-tqdm/issues/31#issuecomment-516325065
tqdm_cb = TQDMNotebookCallback(metric_format="{name}: {value:e}")
# tqdm_cb.on_train_batch_begin = tqdm_cb.on_batch_begin
# tqdm_cb.on_train_batch_end = tqdm_cb.on_batch_end
# tqdm_cb.on_test_begin = lambda x,y:None
# tqdm_cb.on_test_end = lambda x,y:None

In [8]:
model = unet(input_size=(None, 256, 1), lr=1e-3, **run_params)
print(model.summary(line_length=100))

W0919 15:29:23.065008 139772612736768 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



Model: "model_1"
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, 256, 1)  0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, None, 256, 16) 160         input_1[0][0]                    
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, None, 256, 16) 2320        conv2d_1[0][0]                   
____________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)   (None, None, 128, 16) 0           conv2d_2[0][0]                   
__________________________________________________________________________

In [9]:
# %%time
# model.fit_generator(
#     train_gen, 
#     steps_per_epoch=10, 
#     epochs=n_epochs,
#     validation_data=val_gen,
#     validation_steps=1,
#     verbose=0,
#     callbacks=[tqdm_cb, tboard_cback, chkpt_cback,],
# #     max_queue_size=100,
#     use_multiprocessing=True,
#     workers=35,
#     shuffle=False,
# )

In [10]:
%%time
# overfitting trials
data = train_gen[0]
val_data = val_gen[1]
model.fit(
    x=data[0], 
    y=data[1], 
#     validation_data=val_data, 
    batch_size=data[0].shape[0], 
#     callbacks=[tqdm_cb, tboard_cback,],
    epochs=500, 
    verbose=2, 
    shuffle=False,
)

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:139: ComplexWarning: Casting complex values to real discards the imaginary part
  images_scaled = images_scaled.astype(np.float32)
W0919 15:29:27.749298 139772612736768 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Epoch 1/500
 - 10s - loss: 1.0646 - mean_squared_error: 2.7226 - keras_psnr: 13.6051 - keras_ssim: 0.1804
Epoch 2/500
 - 9s - loss: 1.0218 - mean_squared_error: 1.9846 - keras_psnr: 14.9789 - keras_ssim: 0.2464
Epoch 3/500
 - 8s - loss: 0.7610 - mean_squared_error: 1.2959 - keras_psnr: 16.8327 - keras_ssim: 0.2328
Epoch 4/500
 - 8s - loss: 0.7710 - mean_squared_error: 1.1640 - keras_psnr: 17.2969 - keras_ssim: 0.3536
Epoch 5/500
 - 8s - loss: 0.4182 - mean_squared_error: 0.4109 - keras_psnr: 21.8188 - keras_ssim: 0.3795
Epoch 6/500
 - 8s - loss: 0.3704 - mean_squared_error: 0.3362 - keras_psnr: 22.6921 - keras_ssim: 0.4423
Epoch 7/500
 - 8s - loss: 0.3483 - mean_squared_error: 0.3000 - keras_psnr: 23.1869 - keras_ssim: 0.5035
Epoch 8/500
 - 8s - loss: 0.3184 - mean_squared_error: 0.2356 - keras_psnr: 24.2340 - keras_ssim: 0.5750
Epoch 9/500
 - 8s - loss: 0.3222 - mean_squared_error: 0.2258 - keras_psnr: 24.4178 - keras_ssim: 0.5528
Epoch 10/500
 - 8s - loss: 0.3315 - mean_squared_error

KeyboardInterrupt: 